In [16]:
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import networkx as nx
from math import sqrt
import copy
import datetime
from plotly.figure_factory import create_gantt

# 数据预处理

In [17]:
data_a_crew = pd.read_csv('Data_A_Crew.csv')
data_a_flight = pd.read_excel('Data_A_Flight.xlsx')

data_b_crew = pd.read_csv('Data_B_Crew.csv')
data_b_flight = pd.read_csv('Data_B_Flight.csv')

Hou_to_minute = 60
Day_to_hou = 24
Day_to_minute = Day_to_hou * Hou_to_minute

Min_CT = 40
Max_Blk = 600
Max_DP = 720
Min_Rest = 660
Max_DH = 5
Max_TAFB = 14400
Max_SuccOn = 4 * Day_to_minute
Min_VacDay = 2 * Day_to_minute

In [18]:
data_a_flight.head()

,FltNum,DptrDate,DptrTime,DptrStn,ArrvDate,ArrvTime,ArrvStn,Comp
0,FA2,8/12/2021,10:10:00,PGX,8/12/2021,11:40:00,NKX,C1F1
1,FA3,8/12/2021,10:25:00,PGX,8/12/2021,11:40:00,NKX,C1F1
2,FA680,8/11/2021,08:00:00,NKX,8/11/2021,09:30:00,PGX,C1F1
3,FA680,8/12/2021,08:00:00,NKX,8/12/2021,09:30:00,PGX,C1F1
4,FA680,8/13/2021,08:00:00,NKX,8/13/2021,09:30:00,PGX,C1F1


In [19]:
data_a_dptr_date = pd.to_datetime(data_a_flight.DptrDate)
data_a_arrv_date = pd.to_datetime(data_a_flight.ArrvDate)

data_a_dptr_time = data_a_flight.DptrTime.to_list()
data_a_arrv_time = data_a_flight.ArrvTime.to_list()


In [20]:
def get_time_gap(arrv_index, dptr_index):
    arrav_series = data_a_flight.loc[arrv_index]
    dptr_series = data_a_flight.loc[dptr_index]
    arrav_date_time = datetime.datetime.combine(pd.to_datetime(arrav_series.ArrvDate), arrav_series.ArrvTime)
    dptr_date_time = datetime.datetime.combine(pd.to_datetime(dptr_series.DptrDate), dptr_series.DptrTime)
    dateTime_difference = (arrav_date_time - dptr_date_time).total_seconds()/60
    return dateTime_difference
    

In [21]:
get_time_gap(1,0)

90.0

In [22]:
def get_the_time_difference_m(data_dptr_date, data_arrv_date, data_dptr_time, data_arrv_time):
    date_difference = []
    for i in range(len(data_dptr_time)):
        dateTimeA = datetime.datetime.combine(data_dptr_date[i], data_dptr_time[i])
        dateTimeB = datetime.datetime.combine(data_arrv_date[i], data_arrv_time[i])
        dateTime_difference = (dateTimeB - dateTimeA).total_seconds()/60
        date_difference.append(dateTime_difference)
    return date_difference        

In [23]:
data_a_time_difference = get_the_time_difference_m(data_a_dptr_date, data_a_arrv_date, data_a_dptr_time, data_a_arrv_time)

In [24]:
data_a_crew_fill =  data_a_crew.fillna(0)

In [25]:
data_a_crew_fill.FirstOfficer[0] == 0

True

In [26]:
data_a_crew_fill.head()

,EmpNo,Captain,FirstOfficer,Deadhead,Base,Dut1CostPerHour,ParingCostPerHour
0,A0001,1.0,0.0,1,NKX,680,20
1,A0002,1.0,0.0,1,NKX,680,20
2,A0003,1.0,0.0,1,NKX,680,20
3,A0004,1.0,0.0,1,NKX,680,20
4,A0005,1.0,1.0,1,NKX,640,20


In [41]:
# data_a_crew_fill.loc[data_a_crew_fill.EmpNo == 'A0001']
# test_date_time = data_a_flight.loc[data_a_flight.FltNum == 'FA2']
# test_date_ =  test_date_time.Comp.value
# test_set = set()
# data_test = (1,2,3)
# test_set.add(data_test)
# data_test = (3, 4, 5)
# test_set.add(data_test)

# 问题一
## 优化目标
1. 尽可能多航班满主机组配置
2. 尽可能少的总体乘机次数
3. 尽可能少使用替补资格

In [35]:
Set_crew_only_Captain = set()
Set_crew_Captain_FirstOffice = set()
Set_crew_FirstOffice = set()
Set_Flight = set()

Crew_base = "NKX"

for i in range(len(data_a_crew_fill)):
    crew_series = data_a_crew_fill.loc[i]
    if crew_series.Captain == 1:
        if crew_series.FirstOfficer == 1:
            Set_crew_Captain_FirstOffice.add(i)
        else:
            Set_crew_only_Captain.add(i)
    else:
        Set_crew_FirstOffice.add(i)
for i in range(len(data_a_flight)):
    flight_series = data_a_flight.loc[i]
    # 出发地为基地，满足配置直接起飞
    if flight_series.DptrStn == Crew_base:
        if len(Set_crew_only_Captain) != 0:
            captain_temp = Set_crew_only_Captain.pop()
            if len(Set_crew_FirstOffice) != 0:
                firstoffice_temp = Set_crew_FirstOffice.pop()
                temp_array = (i, captain_temp, firstoffice_temp)
                Set_Flight.add(temp_array)
            elif len(Set_crew_Captain_FirstOffice) != 0:
                firstoffice_temp = Set_crew_Captain_FirstOffice.pop()
                temp_array = (i, captain_temp, firstoffice_temp)
                Set_Flight.add(temp_array)
            else:
                Set_crew_only_Captain.add(captain_temp)
        elif len(Set_crew_Captain_FirstOffice) != 0:
            captain_temp = Set_crew_Captain_FirstOffice.pop()
            if len(Set_crew_FirstOffice) != 0:
                firstoffice_temp = Set_crew_FirstOffice.pop()
                temp_array = (i, captain_temp, firstoffice_temp)
                Set_Flight.add(temp_array)
            elif len(Set_crew_Captain_FirstOffice) != 0:
                firstoffice_temp = Set_crew_Captain_FirstOffice.pop()
                temp_array = (i, captain_temp, firstoffice_temp)
                Set_Flight.add(temp_array)
            else:
                Set_crew_Captain_FirstOffice.add(captain_temp)
    else:
        # 目的地为基地
        if flight_series.ArrvStn == Crew_base:
            if len(Set_Flight)


['NKX' 'NKX' 'NKX' 'NKX' 'NKX' 'NKX' 'NKX' 'NKX' 'NKX' 'NKX' 'NKX' 'NKX'
 'NKX' 'NKX' 'NKX']


RangeIndex(start=0, stop=206, step=1)